<a href="https://colab.research.google.com/github/DaryaTereshchenko/AdClick/blob/main/AdClisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pprint import pprint

In [2]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import IntegerLookup, Concatenate
from tensorflow.keras.models import Sequential

In [3]:
# load data
from urllib.request import urlretrieve
import os

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

d100k_url = "https://home.ipipan.waw.pl/sj/Warsztaty/AdClick/D10M.tsv.gz"
download(d100k_url,'D100k.tsv.gz')
print("All the files are downloaded")

Download file... D100k.tsv.gz ...
File downloaded
All the files are downloaded


In [4]:
ds = tf.data.experimental.make_csv_dataset("D100k.tsv.gz",
                                           field_delim="\t",
                                           compression_type="GZIP",
                                           num_epochs=1,
                                           batch_size=128,  # increase for real work
                                           label_name="Click", shuffle=False)


In [5]:
r=next(iter(ds))

In [12]:
r[0]["Age"]

<tf.Tensor: shape=(128,), dtype=int32, numpy=
array([3, 3, 3, 2, 3, 3, 3, 4, 3, 1, 1, 6, 6, 4, 4, 4, 3, 3, 1, 1, 1, 6,
       1, 4, 3, 3, 4, 1, 1, 1, 4, 3, 6, 5, 6, 4, 1, 6, 3, 3, 3, 3, 6, 6,
       4, 4, 3, 3, 5, 3, 6, 6, 4, 4, 3, 6, 4, 3, 3, 4, 4, 1, 1, 1, 3, 3,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 4,
       3, 3, 6, 4, 4, 3, 3, 3, 3, 2, 2, 4, 4, 4, 4, 1, 1, 1, 4, 4, 4, 3,
       3, 3, 3, 3, 3, 3, 2, 3, 3, 4, 3, 3, 3, 2, 3, 1, 1, 3], dtype=int32)>

In [23]:
inputs = {
    "Position": Input(shape=(), dtype=tf.int32),
    "Age": Input(shape=(), dtype=tf.int32),
    "Depth": Input(shape=(), dtype=tf.int32),
    "Gender": Input(shape=(), dtype=tf.int32),
}

input_age = IntegerLookup(vocabulary=[1,2,3,4,5,6], output_mode="one_hot", num_oov_indices=0)(inputs["Age"])
ci_p = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Position"])
ci_d = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Depth"])
ci_g = IntegerLookup(vocabulary=[0,1,2], output_mode="one_hot", num_oov_indices=0)(inputs["Gender"])

In [24]:
encoded = Concatenate()([ci_p, input_age, ci_d, ci_g])

In [25]:
x = Dense(2, activation="relu")(encoded)
y = Dense(1, activation="sigmoid")(x)
m1 = Model(inputs, y)

In [26]:
m1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None,)]            0           []                               
                                                                                            

In [27]:
m1.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
          metrics=["AUC"])

In [29]:
m1.fit(ds.unbatch().batch(128), epochs=5)

Epoch 1/5
78125/78125 [==============================] - 387s 5ms/step - loss: 0.1861 - auc: 0.6030
Epoch 2/5
78125/78125 [==============================] - 383s 5ms/step - loss: 0.1861 - auc: 0.6030
Epoch 3/5
78125/78125 [==============================] - 384s 5ms/step - loss: 0.1861 - auc: 0.6030
Epoch 4/5
78125/78125 [==============================] - 384s 5ms/step - loss: 0.1861 - auc: 0.6030
Epoch 5/5
78125/78125 [==============================] - 384s 5ms/step - loss: 0.1861 - auc: 0.6030
